In [1]:
    import os
import base64
from PIL import Image
from io import BytesIO
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import warnings

In [2]:
def get_image_file_paths(directory):
    # 이미지 파일 확장자 목록
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
    
    # 디렉토리 내의 이미지 파일 경로를 저장할 리스트
    image_files = []
    
    # 디렉토리 내의 모든 파일을 검사
    for filename in os.listdir(directory):
        # 파일의 전체 경로를 생성
        file_path = os.path.join(directory, filename)
    
        # 파일이 디렉토리가 아닌 경우에만 처리
        if os.path.isfile(file_path):
            # 파일의 확장자를 확인
            if any(filename.lower().endswith(ext) for ext in image_extensions):
                image_files.append(file_path)
    
    return image_files

# 사용 예시
directory_path = 'image_data'
image_file_paths = get_image_file_paths(directory_path)


In [3]:
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        
        img = Image.open(image_file)
        img = img.resize((1000, 1000))
        buffer = BytesIO()
        img.save(buffer, format="PNG")

        return base64.b64encode(buffer.getvalue()).decode("utf-8")
        
        
        

# PDF 요소에서 이미지 경로를 가져옴
encoded_image_list = list()

for path in image_file_paths:
    image_data = encode_image_to_base64(path)
    encoded_image_list.append(image_data)

In [12]:
print(len(encoded_image_list))

13


In [5]:
# input open ai api key 
api_key = "개인 api key"

# chatbot persona
한국어 버전 : 다음 표 형식의 데이터를 자연스럽고 일관된 서술형 문장으로 변환해 주세요. 표의 구조를 유지하지 말고, 각 항목의 의미를 명확하게 설명하도록 변환해 주세요. 각 항목 간의 관계를 정확히 표현하면서, 검색 시스템이 각 개념을 정확히 이해할 수 있도록 중요 개념을 강조해 주세요. 표에 담긴 정보를 잃지 않고, 명확한 문장으로 풀어서 설명하는 방식으로 변환해 주세요. 표 형식의 틀을 유지하지 말고, 서술형으로 변환하는 데 집중하세요. 예를 들어, 인물, 주장, 모델 같은 부분은 자연스럽게 연결하여 서술하고, 의미 전달이 중요한 부분은 중점적으로 서술해 주세요. 또한 각 개념이 나중에 유사한 쿼리로 검색될 때 혼동을 최소화할 수 있도록 하세요

영어 버전 : Please transform the following table data into a natural and consistent narrative format. Do not preserve the table structure, but instead, explain the meaning of each entry clearly in sentence form. Be sure to accurately express the relationships between the different entries, so that the search system can understand each concept correctly. Focus on turning the table’s structure into coherent and clear sentences without losing any of the information contained in the table. Avoid maintaining the table format, and instead, prioritize converting the information into a descriptive format. For example, for entries like 'person,' 'argument,' or 'model,' connect them naturally into a sentence and emphasize the key points. Ensure that the main concepts are well-explained and clearly articulated, to avoid confusion when searching for similar queries in the future. Answer in KOREAN.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", openai_api_key = api_key, temperature=0.3)
model2 = ChatOpenAI(model="gpt-4o-mini", openai_api_key = api_key, temperature=0.4)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Avoid including introductory phrases such as 'This document is' or 'This is a document about. Please transform the following table data into a natural and consistent narrative format. Do not preserve the table structure, but instead, explain the meaning of each entry clearly in sentence form. Be sure to accurately express the relationships between the different entries, so that the search system can understand each concept correctly. Focus on turning the table’s structure into coherent and clear sentences without losing any of the information contained in the table. Avoid maintaining the table format, and instead, prioritize converting the information into a descriptive format. For example, for entries like 'person,' 'argument,' or 'model,' connect them naturally into a sentence and emphasize the key points. Ensure that the main concepts are well-explained and clearly articulated, to avoid confusion when searching for similar queries in the future. Answer in KOREAN."),
        (
            "user",
            [
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                }
            ],
        ),
    ]
)


chain = prompt | model

In [14]:
response_list = list()

for image_data in encoded_image_list:
    response = chain.invoke({"image_data": image_data})
    response_list.append(response.content)




In [15]:
print(response_list[4])

위항 상인 운동의 전개는 18세기 군현 소재지의 양반 출신 상인들이 상업적 세력을 키워나가는 과정에서 비롯되었다. 이들은 위항 상인으로 불렸다. 18세기에 상업이 발달하고 서구 문물이 유입되면서 위항 상인들은 주로 중국과의 무역을 통해 부를 축적하였다. 이들은 청나라와의 무역을 통해 사치품을 수입하고, 국내에서는 쌀, 인삼, 면포 등을 판매하였다. 19세기에는 개항 이후 일본과의 무역이 활발해지면서 위항 상인들의 활동 범위가 더욱 넓어졌다. 이들은 일본과의 무역을 통해 면직물, 잡화 등을 수입하고, 국내에서는 쌀, 인삼, 면포 등을 판매하였다. 이러한 과정에서 위항 상인들은 상업적 세력을 키워나갔고, 이를 바탕으로 정치적 영향력도 행사하게 되었다.

위항 상인 운동의 흐름은 크게 통상적 단계, 개항 전 단계, 개항 후 단계로 나눌 수 있다. 통상적 단계에서는 위항 상인들이 주로 중국과의 무역을 통해 부를 축적하였다. 이들은 청나라와의 무역을 통해 사치품을 수입하고, 국내에서는 쌀, 인삼, 면포 등을 판매하였다. 개항 전 단계에서는 위항 상인들이 주로 국내 상업 활동을 통해 부를 축적하였다. 이들은 국내에서 쌀, 인삼, 면포 등을 판매하고, 이를 통해 부를 축적하였다. 개항 후 단계에서는 위항 상인들이 주로 일본과의 무역을 통해 부를 축적하였다. 이들은 일본과의 무역을 통해 면직물, 잡화 등을 수입하고, 국내에서는 쌀, 인삼, 면포 등을 판매하였다.

임오군란은 개항 이후 조선의 양반 출신 상인들이 아닌 구식 군대의 군인들이 도시 하층민들과 더불어 일으킨 반란이다. 정부의 개혁 시도에 불만을 품은 이들은 일본과의 무역으로 인해 경제적 어려움을 겪고 있었다. 이에 따라 1882년 임오군란이 발생하였고, 이는 조선 정부의 개혁 시도에 큰 영향을 미쳤다. 임오군란의 주요 원인은 구식 군대의 군인들이 일본과의 무역으로 인해 경제적 어려움을 겪고 있었기 때문이다. 이들은 일본과의 무역으로 인해 경제적 어려움을 겪고 있었고, 이에 따라 정부의 개혁 시도에 불만을 품게 되었

In [41]:
'''
이 문서는 근대 국민 국가 수립 운동과 동아시아의 변화 및 근대적 개혁의 추진에 관한 내용입니다. 특히 갑신정변이 일어나고 열강의 대립이 커진 상황을 설명하고 있습니다.

갑신정변의 주도 세력은 온건 개화파와 급진 개화파로 나눌 수 있습니다. 온건 개화파에는 김윤식, 김홍집, 어윤중이 있었으며, 개혁 모델로는 청의 양무운동을 따랐습니다. 이들은 청도 서구를 본받아야 한다고 주장했습니다. 반면 급진 개화파에는 김옥균, 박영효, 홍영식이 있었고, 이들은 일본의 메이지 유신을 개혁 모델로 삼았습니다. 이들은 문명 개화론을 주장했으며, 서양 기술은 수용하되 사상은 수용하지 않는 입장을 취했습니다.

갑신정변의 배경으로는 청의 책문과 일본에서 차관 도입 실패로 급진 개화파의 정치적 입지가 약해졌으며, 청-프 전쟁에서 청이 조선에 주둔한 군대를 철수하면서 일본이 군사적 지원을 약속한 상황이 있었습니다.

정변은 급진 개화파와 우정총국 개국 축하연을 이용해 정변을 일으켰으며, 민씨 세력 제거와 개화당 정부 구성을 목표로 했습니다. 개화 정강을 발표하여 청에 조공하는 허례를 폐지하고, 문벌 폐지와 인민 평등권을 제정하며, 재정을 호조에 관할하고 박외 재무 관청을 폐지하고, 개국한 상업을 호조로 일원화하는 등의 개혁을 추진했습니다. 또한 대소 민관은 의정부에서 회의하여 국왕에게 아뢰는 정령을 전달하자는 등의 내각 제도 수립을 주장했습니다.

정변의 진압은 삼일 천하로 끝났습니다. 그 결과 조선-일본 간 한성 조약이 체결되어 조선이 일본에게 배상금과 공사관 신축 비용을 지불해야 했습니다. 청-일본 간 텐진 조약이 체결되어 조선에서 청과 일본의 군대가 철수하고 조선에 파병 시 상호국에 미리 통보하도록 했습니다.

이 의의는 자주적 근대 국가 건설을 위한 우리나라 최초의 정치 개혁 운동이라는 점에 있습니다. 하지만 한계로는 소수의 지식인을 중심으로 한 개혁으로 민중의 지지를 얻지 못했으며, 일본의 군사적 지원에 지나치게 의존했다는 점이 있습니다.













갑신정변은 1884년에 일어난 사건으로, 온건 개화파와 급진 개화파로 나뉘어 진행되었습니다. 온건 개화파에는 김윤식, 김홍집, 어윤중이 포함되었으며, 그들의 개혁 모델은 청의 양무운동이었습니다. 이들은 청도 서구로부터 배워야 한다고 주장했습니다. 반면, 급진 개화파에는 김옥균, 박영효, 홍영식이 포함되었으며, 일본의 메이지 유신을 개혁 모델로 삼았습니다. 이들은 문명 개화론을 주장하며, 전통적인 서양 기술과 사상을 받아들여야 한다고 보았습니다.

갑신정변의 배경으로는 일본에서 차관 도입 실패와 급진 개화파의 정치적 입지가 있었습니다. 또한, 청-프 전쟁에서 청이 조선에 주둔한 군대를 철수하고 일본이 군사적 지원을 약속한 점도 중요한 배경이 되었습니다.

전개 과정에서 급진 개화파는 우정총국 개국 축하연을 이용해 정변을 일으켰습니다. 이들은 민씨 세력을 제거하고 개화당 정부를 구성하며 개혁 정강을 발표했습니다. 주요 내용으로는 청에 대한 사대관계 청산, 인민 평등권 제정, 재정 호조로의 일원화, 내각 제도 수립 등이 있었습니다. 그러나 정변은 '3일 천하'로 끝나고 말았습니다.

결과적으로 조선과 일본은 한성조약을 체결하게 되었고, 조선은 일본에게 배상금과 공사관 신축 비용을 지불하게 되었습니다. 청일본은 텐진 조약을 체결하여 조선에서 청과 일본 군대가 철수하고, 조선에 파병 시 상대국에 미리 통보하기로 했습니다. 갑신정변은 자주적 근대 국가 건설을 위한 우리나라 최초의 정치 개혁 운동으로 의의가 있습니다.

하지만 한계도 존재했습니다. 소수의 지식인들 중심의 개혁으로 민중의 지지를 받지 못했고, 일본의 군사적 지원에 지나치게 의존한 점이 문제로 지적되었습니다.










개화파는 온건 개화파와 급진 개화파로 나뉩니다. 온건 개화파의 주요 인물로는 김윤식, 김홍집, 어윤중이 있으며, 급진 개화파의 주요 인물로는 김옥균, 박영효, 홍영식이 있습니다. 온건 개화파는 청의 양무운동을 개혁 모델로 삼았고, 급진 개화파는 일본의 메이지 유신을 모델로 삼았습니다. 온건 개화파는 청도 서구로, 정/서양 기술, 사상 X의 주장을 펼쳤고, 급진 개화파는 문명 개화론, 정/서양 기술, 사상 O의 주장을 펼쳤습니다.

배경으로는 조선이 일본에서 차관 도입에 실패하면서 급진 개화파가 정치적 입지를 잃게 되었고, 청-프 전쟁에서 청이 조선에 주둔한 군대를 철수하면서 일본이 군사적 지원을 약속하게 되었습니다. 급진 개화파는 우정총국 개국 축하연을 이용해 정변을 일으켰고, 민씨 세력을 제거하고 개화당 정부를 구성하며 개혁 정강을 발표했습니다.

정강의 주요 내용은 다음과 같습니다. 청에 대한 사대관계를 청산하고, 문벌 폐지와 인민 평등권을 제정하며, 능력에 따른 관리 임명, 인민의 평등권 보장, 재정을 모두 호조에서 관할하고 박외 재무 관청을 폐지, 혜상공국을 폐지하고 자유로운 상업 활동 보장, 대신과 참찬은 의정부에서 회의하고 국왕에게 아뢸 것, 내각 제도 수립 등이 있습니다.

정변의 진압은 '3일 천하'로 끝났습니다. 결과적으로 조선과 일본은 한성조약을 체결하여 조선이 일본에게 배상금과 공사관 신축 비용을 지불하게 되었고, 청과 일본은 텐진 조약을 체결하여 조선에서 청과 일본 군대가 철수하고 조선에 파병 시 상대국에 미리 통보하게 되었습니다.

의의로는 자주적 근대 국가 건설을 위한 우리나라 최초의 정치 개혁 운동이었으며, 한계로는 소수의 지식인들 중심의 개혁으로 민중의 지지를 받지 못했고, 일본의 군사적 지원에 지나치게 의존했다는 점이 있습니다.








이 문서는 근대 국민 국가 수립 운동 중 동아시아의 변화와 근대적 개혁의 추진에 관한 내용을 다루고 있습니다. 특히 갑신정변이 일어나고 열강의 대립이 커진 시기에 대해 설명하고 있습니다.

개화파는 온건 개화파와 급진 개화파로 나뉩니다. 온건 개화파에는 김윤식, 김홍집, 어윤중이 속하며, 그들의 개혁 모델은 청의 양무운동입니다. 이들은 청도 서구로 보았으며, 정/서양 기술을 수용하고 사상은 받아들이지 않았습니다. 반면, 급진 개화파에는 김옥균, 박영효, 홍영식이 속하며, 일본의 메이지 유신을 개혁 모델로 삼았습니다. 이들은 문명 개화론을 주장하며, 정/서양 기술과 사상을 모두 수용했습니다.

갑신정변의 배경으로는 조선이 일본에서 차관 도입에 실패하고, 정치적 입지에 있던 급진 개화파가 청-프 전쟁에서 조선에 주둔한 군대 철수와 일본의 군사적 지원 약속을 받았다는 점이 있습니다.

전개 과정에서 급진 개화파는 우정총국 개국 축하연을 이용해 정변을 일으켰고, 민씨 세력을 제거하고 개화당 정부를 구성했습니다. 개혁 정강을 발표하며 청에 조공하는 허례를 폐지하고, 문벌 폐지와 인민 평등권을 제정하며, 재정을 호조에 일원화하고, 대신과 참찬은 의정부에서 회의하여 정령을 반포하는 등의 내용을 포함했습니다. 그러나 정변은 3일 천하로 끝났습니다.

결과적으로 조선과 일본은 한성조약을 체결하였고, 조선이 일본에게 배상금과 공사관 신축 비용을 지불하게 되었습니다. 청일본은 텐진 조약을 체결하여 조선에서 정, 일본 군대 철수와 조선에 파병 시 상호 통보를 약속했습니다.

갑신정변의 의의는 자주적 근대 국가 건설을 위한 우리나라 최초의 정치 개혁 운동이라는 점입니다. 하지만 한계로는 소수의 지식인들 중심의 개혁으로 민중의 지지를 얻지 못했고, 일본의 군사적 지원에 지나치게 의존했다는 점이 있습니다.
'''

SyntaxError: invalid syntax (391952976.py, line 2)

In [23]:
print(response_list[0])

갑신정변은 1884년에 발생한 사건으로, 개화파와 급진 개화파로 나뉘어 진행되었습니다. 개화파에는 김윤식, 김홍집, 어윤중이 포함되었으며, 개화의 모델로 청나라의 양무운동을 따랐습니다. 이들은 온건 개화파로 분류되며, 서양 기술의 수용에는 찬성했으나 사상 수용에는 반대했습니다. 급진 개화파에는 김옥균, 박영효, 홍영식이 포함되었으며, 일본의 메이지 유신을 모델로 삼았습니다. 이들은 급진 개화파로 분류되며, 서양 기술과 사상의 수용에 모두 찬성했습니다.

갑신정변의 배경으로는 청나라의 개입과 일본에서의 서구화 도입 실패, 그리고 조선에서의 정치적 불안정이 있었습니다. 정변의 전개 과정에서는 우정총국 개국 축하연을 이용한 정변이 발생했고, 개화당 정부가 수립되었습니다. 개화당은 홍영식을 통해 군대를 동원하여 궁궐을 점령하고 개화 정책을 발표했습니다. 주요 개혁 내용으로는 청나라에 대한 조공 허례 폐지, 문벌 폐지와 인민 평등권 제정, 지조법 개혁, 내시부 폐지, 탐관오리 처벌, 환상미 영구 면제, 규장각 폐지, 순사 제도 실시, 혜상공국 혁파, 재정 일원화, 근위대 설치 등이 있었습니다.

그러나 정변은 3일 천하로 끝났으며, 청나라의 개입으로 인해 실패하게 되었습니다. 정변 이후 조선은 일본과 텐진 조약을 체결하고, 청나라의 내정 간섭이 심화되었으며, 일본 공사관 신축 비용을 지불하게 되었습니다. 또한, 조선에서의 군대 개편이 이루어졌고, 일본의 경제적 침투가 가속화되었습니다. 갑신정변의 의의는 근대 국민 국가 수립 운동의 선구적 역할을 했다는 점과, 이후 우리나라의 정치 개혁 운동에 영향을 미쳤다는 점입니다. 하지만 민중의 지지를 얻지 못하고 일본의 후원자 역할에 그쳤다는 한계도 있었습니다.


In [17]:
chunked_text_list = list()

c_splitter = CharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=0,
    separator = '\n\n'
)


for response_text in response_list:
    chunked_text = c_splitter.split_text(response_text)
    chunked_text_list.append(chunked_text)
    
    

Created a chunk of size 254, which is longer than the specified 30
Created a chunk of size 302, which is longer than the specified 30
Created a chunk of size 174, which is longer than the specified 30
Created a chunk of size 398, which is longer than the specified 30
Created a chunk of size 185, which is longer than the specified 30
Created a chunk of size 106, which is longer than the specified 30
Created a chunk of size 202, which is longer than the specified 30
Created a chunk of size 126, which is longer than the specified 30
Created a chunk of size 162, which is longer than the specified 30
Created a chunk of size 173, which is longer than the specified 30
Created a chunk of size 161, which is longer than the specified 30
Created a chunk of size 291, which is longer than the specified 30
Created a chunk of size 366, which is longer than the specified 30
Created a chunk of size 340, which is longer than the specified 30
Created a chunk of size 359, which is longer than the specifie

In [18]:
print(len(chunked_text_list))

13


In [36]:
chunked_text_list[5]

['간신전쟁이 일어나고 열강의 대립이 커지다',
 '임진왜란이 끝난 이후 조선은 국토와 경제에서 큰 피해를 입었고, 이를 복구하기 위해 많은 노력을 기울였다. 이 시기에 조선은 국방력을 강화하고, 외교적으로도 주변국들과의 관계를 재정립하는 데 힘썼다. 그러나 이러한 노력에도 불구하고 조선은 여러 차례의 외세 침략을 겪게 된다.',
 '간신전쟁의 전개',
 '조선은 러시아와의 국경 분쟁으로 인해 간신전쟁을 겪게 되었다. 이 전쟁은 러시아의 남하 정책과 조선의 국경 방어 정책이 충돌하면서 발생하였다. 조선은 러시아의 침략을 막기 위해 여러 차례 전투를 벌였고, 이 과정에서 많은 희생이 있었다. 전쟁이 끝난 후 조선은 국경을 재정비하고, 군사력을 강화하는 데 집중하였다.',
 '간신전쟁의 영향',
 '간신전쟁은 조선 사회에 큰 영향을 미쳤다. 전쟁으로 인해 많은 인명 피해와 재산 손실이 발생하였고, 이는 조선의 경제와 사회 구조에 큰 변화를 가져왔다. 또한, 전쟁을 통해 조선은 군사력의 중요성을 다시 한 번 깨닫게 되었고, 이를 바탕으로 국방력을 강화하는 정책을 추진하게 되었다.',
 '전쟁 이후 조선은 외교적으로도 많은 변화를 겪었다. 주변국들과의 관계를 재정립하고, 외교 정책을 강화하는 데 힘썼다. 이를 통해 조선은 외세의 침략에 대비하고, 자주적인 국가로서의 입지를 다지게 되었다.',
 '간신전쟁의 교훈',
 '간신전쟁은 조선에게 많은 교훈을 남겼다. 전쟁을 통해 조선은 국방력의 중요성을 깨닫게 되었고, 이를 바탕으로 군사력을 강화하는 정책을 추진하게 되었다. 또한, 외교적으로도 주변국들과의 관계를 재정립하고, 외교 정책을 강화하는 데 힘썼다. 이를 통해 조선은 외세의 침략에 대비하고, 자주적인 국가로서의 입지를 다지게 되었다.',
 '이와 같은 간신전쟁의 경험은 조선이 이후에도 외세의 침략에 대비하고, 자주적인 국가로서의 입지를 다지는 데 중요한 역할을 하였다.']

In [21]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)

C:\Users\qa762\AppData\Local\Temp\ipykernel_15684\2980451733.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)


In [22]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

documents_list = list()

for doc in chunked_text_list:
    documents = [Document(page_content=text) for text in doc]
    documents_list.append(documents)
    

vectorstore = FAISS.from_documents(documents_list[0], embeddings)


for data in documents_list[1:]:
    vectorstore.add_documents(data)



In [52]:
vectorstore

In [46]:
query_text = "조선은 러시아와의 국경 분쟁으로 인해 간신전쟁을 겪게 되었다."

### 문제점 1. 청크 사이즈가 작은 것들을 큰 문장에 합쳐야함

In [47]:
docs = vectorstore.similarity_search(query_text, k=2)
docs

[Document(metadata={}, page_content='조선은 러시아와의 국경 분쟁으로 인해 간신전쟁을 겪게 되었다. 이 전쟁은 러시아의 남하 정책과 조선의 국경 방어 정책이 충돌하면서 발생하였다. 조선은 러시아의 침략을 막기 위해 여러 차례 전투를 벌였고, 이 과정에서 많은 희생이 있었다. 전쟁이 끝난 후 조선은 국경을 재정비하고, 군사력을 강화하는 데 집중하였다.'),
 Document(metadata={}, page_content='청일 전쟁과 삼국 간섭에서는 청나라와 일본이 조선의 지배권을 두고 전쟁을 벌였으며, 일본이 승리하게 되었다. 그러나 이후 러시아, 독일, 프랑스가 일본의 조선 지배를 견제하기 위해 삼국 간섭을 벌였다.')]

In [62]:
dir_path = "image_data"

len(dir_path)

In [48]:
vectorstore.save_local('vector_db')